In [110]:
# 预计爬取前20页书名，评分，价格，目前状态
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as  np

In [108]:
# names = locals()
# for i in range(1,20):
#    names['respone' + str(i) ] = requests.get( "https://books.toscrape.com/catalogue/page-{}.html".format(i))

In [68]:
def bsf_bkname(respone):
    soup = BeautifulSoup(respone.content, 'html.parser')
    layer1 = soup.find_all(class_='product_pod')
    book_name_list_html = []
    for i in layer1:
        book_name_list_html.append(i.h3)
    book_name_list =[]
    for j in book_name_list_html:
        book_name_list.append(j.a.get('title'))
    return book_name_list

In [87]:
def bsf_bkprice(respone):
    soup = BeautifulSoup(respone.content, 'html.parser')
    layer1 = soup.find_all(class_='price_color')
    book_price = []
    for i in layer1:
        book_price.append(i.get_text())
    return book_price

In [101]:
def bsf_bkstatus(respone):
    soup = BeautifulSoup(respone.content, 'html.parser')
    layer1 = soup.find_all(class_='instock availability')
    book_status = []
    for i in layer1:
        book_status.append(i.get_text(strip='True'))
    return book_status

In [102]:
def crawlprocess():
    booklist ={}
    bookpricelist = {}
    bookstatus = {}
    for page in range(1,23):
        print('crawling page {} now'.format(page))
        url = "https://books.toscrape.com/catalogue/page-{}.html".format(page)
        headers={
            "User-Agent":"USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36"
        }
        data = {
            "City":None,
            "pageIndex":page,
        }
        respone = requests.get(url = url, headers = headers, data = data)
        booklist[str(page)] = bsf_bkname(respone)
        bookpricelist[str(page)] = bsf_bkprice(respone)  
        bookstatus[str(page)] = bsf_bkstatus(respone)                        
        time.sleep(3)
    return booklist,bookpricelist,bookstatus

In [104]:
bk_name_lst,bk_price_list,bk_status_list= crawlprocess()

crawling page 1 now
crawling page 2 now
crawling page 3 now
crawling page 4 now
crawling page 5 now
crawling page 6 now
crawling page 7 now
crawling page 8 now
crawling page 9 now
crawling page 10 now
crawling page 11 now
crawling page 12 now
crawling page 13 now
crawling page 14 now
crawling page 15 now
crawling page 16 now
crawling page 17 now
crawling page 18 now
crawling page 19 now
crawling page 20 now
crawling page 21 now
crawling page 22 now


In [123]:
#创建一个dataframe拼接
def get_data(dic):
    data = pd.DataFrame(dic)
    df_new = pd.DataFrame(data.to_numpy().reshape(-1, 1, order='F'), columns=['1'])
    return df_new

In [124]:
# 创建多列拼接
def concate_data(*diclist):
    data1 = diclist[0]
    for i in diclist[1:]:
        data1 = pd.concat([data1, i],axis=1,ignore_index=True)
    return data1


In [125]:
data = concate_data(get_data(bk_name_lst),get_data(bk_price_list),get_data(bk_status_list))

In [136]:
data = data.rename(columns={0:'name',1:'price',2:'status'})

In [137]:
data

,name,price,status
0,A Light in the Attic,£51.77,In stock
1,Tipping the Velvet,£53.74,In stock
2,Soumission,£50.10,In stock
3,Sharp Objects,£47.82,In stock
4,Sapiens: A Brief History of Humankind,£54.23,In stock
...,...,...,...
435,Modern Day Fables,£47.44,In stock
436,If I Gave You God's Phone Number....: Searchin...,£20.91,In stock
437,"Fruits Basket, Vol. 9 (Fruits Basket #9)",£33.95,In stock
438,Dress Your Family in Corduroy and Denim,£43.68,In stock


In [138]:
data['price'] = data['price'].apply(lambda x: x[1:])

In [139]:
data.to_csv(r'C:\Users\22785\Desktop\cuhk course\term3\web\scpdata.csv')